In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback, ProgbarLogger
from tensorflow.keras import regularizers as R
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import layers as L
from tensorflow.keras import optimizers as O
from tensorflow.keras import constraints as C
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy, Loss
from tensorflow.keras.metrics import AUC
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score
tf.keras.utils.set_random_seed(722)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
def balanced_log_loss_np(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    nc = np.bincount(y_true)
    balanced_log_loss_score = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / 2
    return balanced_log_loss_score

In [3]:
train_df = pd.read_csv('data/train.csv', index_col='Id')

train_df['EJ'] = train_df['EJ'].replace({'A': 0, 'B': 1})

nan_fill = train_df.isna().any()
nan_fill *= train_df.min() - train_df.max()
nan_fill[nan_fill == 0] = train_df.median()
train_df = train_df.fillna(nan_fill)

In [4]:
X = train_df.iloc[:,:-1].values
tgt = train_df.Class.values

In [5]:
### This is the hard-coded label from baseline model, where "1" is difficult to predict, "0" - easy.
### (y_true = 1 and y_pred < 0.2) or (y_true = 0 and y_pred > 0.8) -> label "1", otherwise label "0".
tgt2 = np.asarray([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0], dtype=np.int32)

In [8]:
@tf.keras.utils.register_keras_serializable()
def smish(x):
    return x * K.tanh(K.log(1 + K.sigmoid(x)))


@tf.keras.utils.register_keras_serializable()
class GatedLinearUnit(L.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.linear = L.Dense(units)
        self.sigmoid = L.Dense(units, activation="sigmoid")
        self.units = units

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        return config
    
    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)
    

@tf.keras.utils.register_keras_serializable()
class GatedResidualNetwork(L.Layer):
    def __init__(self, units, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.dropout_rate = dropout_rate
        self.relu_dense = L.Dense(units, activation=smish)
        self.linear_dense = L.Dense(units)
        self.dropout = L.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = L.LayerNormalization()
        self.project = L.Dense(units)

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        x = self.relu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x
    

@tf.keras.utils.register_keras_serializable()
class VariableSelection(L.Layer):
    def __init__(self, num_features, units, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = L.Dense(units=num_features, activation="softmax")
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate

    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        v = L.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input_ in enumerate(inputs):
            x.append(self.grns[idx](input_))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
    

@tf.keras.utils.register_keras_serializable()
class VariableSelectionFlow(L.Layer):
    def __init__(self, num_features, units, dropout_rate, dense_units=None, **kwargs):
        super().__init__(**kwargs)
        self.variableselection = VariableSelection(num_features, units, dropout_rate)
        self.split = L.Lambda(lambda t: tf.split(t, num_features, axis=-1))
        self.dense = dense_units
        if dense_units:
            self.dense_list = [L.Dense(dense_units, \
                                       activation='linear') \
                               for _ in tf.range(num_features)
                              ]
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate
        self.dense_units = dense_units
        
    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        config['dense_units'] = self.dense_units
        return config        
    
    def call(self, inputs):
        split_input = self.split(inputs)
        if self.dense:
            l = [self.dense_list[i](split_input[i]) for i in range(len(self.dense_list))]
        else:
            l = split_input
        return self.variableselection(l)        

In [9]:
%%time

blls = []

batch_size = 32

units_1 = 32
drop_1 = 0.75
dense_units = 8

units_2 = 16
drop_2 = 0.5

units_3 = 8
drop_3 = 0.25

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=722)
for n, (train_idx, val_idx) in enumerate(cv.split(X, (tgt + 1) * (tgt2 - 3))):
    for k in range(10):
        print(f'______fold {n+1}______, ________repeat {k+1}__________')

        inputs_1 = tf.keras.Input(shape=(56,))
        
        features_1 = VariableSelectionFlow(56, units_1, drop_1, dense_units=dense_units)(inputs_1)
        features_2 = VariableSelectionFlow(units_1, units_2, drop_2)(features_1)         
        features_3 = VariableSelectionFlow(units_2, units_3, drop_3)(features_2)         

        outputs = L.Dense(1, activation="sigmoid")(features_3)

        model = Model(inputs=inputs_1, outputs=outputs)      

        opt = O.Adam(1e-3, epsilon=1e-7)
        loss = binary_crossentropy

        lr = ReduceLROnPlateau(monitor="val_loss", mode='min', factor=0.95, patience=1, verbose=1)
        es = EarlyStopping(monitor='val_loss', mode='min', patience=25, verbose=1, restore_best_weights=True)

        model.compile(optimizer=opt, 
                        loss=loss
                     )

        history = model.fit(x=X[train_idx],
                          y=tgt[train_idx],
                          batch_size=batch_size,
                          epochs=200,
                          validation_data=(X[val_idx], tgt[val_idx]),
                              callbacks=[lr,es]
                )
                
        probs = model.predict(X[val_idx])[:,0]
        bll = balanced_log_loss_np(train_df.Class.values[val_idx], probs)
        blls.append(bll)
        val_loss = np.asarray(history.history['val_loss'])
        train_loss = np.asarray(history.history['loss'])
        min_val_loss = val_loss.min()
        min_train_loss = train_loss[val_loss.argmin()]
        print(f'{min_train_loss:.4f}, {min_val_loss:.4f}, {bll:.4f}')  
        
        model.save_weights(f'ICR_tf_adv_models/mod_f{n}_r{k}_tr{min_train_loss:.4f}_val{min_val_loss:.4f}.h5')
        
print(np.mean(blls))

______fold 1______, ________repeat 1__________
Epoch 1/200
18/18 [==============================] - 50s 769ms/step - loss: 0.4855 - val_loss: 0.4567 - lr: 0.0010
Epoch 2/200
18/18 [==============================] - ETA: 0s - loss: 0.4414
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
18/18 [==============================] - 7s 391ms/step - loss: 0.4414 - val_loss: 0.4646 - lr: 0.0010
Epoch 3/200
18/18 [==============================] - 7s 397ms/step - loss: 0.4400 - val_loss: 0.4551 - lr: 9.5000e-04
Epoch 4/200
18/18 [==============================] - 7s 397ms/step - loss: 0.4144 - val_loss: 0.4449 - lr: 9.5000e-04
Epoch 5/200
18/18 [==============================] - 7s 396ms/step - loss: 0.4107 - val_loss: 0.4375 - lr: 9.5000e-04
Epoch 6/200
18/18 [==============================] - ETA: 0s - loss: 0.3943
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0009025000152178108.
18/18 [==============================] - 7s 400ms/step - loss: 0.3943 - val_lo

18/18 [==============================] - 7s 368ms/step - loss: 0.1710 - val_loss: 0.2766 - lr: 2.9199e-04
Epoch 41/200
18/18 [==============================] - ETA: 0s - loss: 0.1868
Epoch 41: ReduceLROnPlateau reducing learning rate to 0.0002635200624354184.
18/18 [==============================] - 7s 366ms/step - loss: 0.1868 - val_loss: 0.2645 - lr: 2.7739e-04
Epoch 42/200
18/18 [==============================] - ETA: 0s - loss: 0.1754
Epoch 42: ReduceLROnPlateau reducing learning rate to 0.0002503440482541919.
18/18 [==============================] - 7s 364ms/step - loss: 0.1754 - val_loss: 0.2740 - lr: 2.6352e-04
Epoch 43/200
18/18 [==============================] - ETA: 0s - loss: 0.1767
Epoch 43: ReduceLROnPlateau reducing learning rate to 0.00023782684584148226.
18/18 [==============================] - 7s 372ms/step - loss: 0.1767 - val_loss: 0.2786 - lr: 2.5034e-04
Epoch 44/200
18/18 [==============================] - ETA: 0s - loss: 0.1784
Epoch 44: ReduceLROnPlateau reducing

Epoch 20/200
18/18 [==============================] - 7s 390ms/step - loss: 0.1994 - val_loss: 0.2175 - lr: 6.3025e-04
Epoch 21/200
18/18 [==============================] - ETA: 0s - loss: 0.1981
Epoch 21: ReduceLROnPlateau reducing learning rate to 0.0005987368611386045.
18/18 [==============================] - 7s 367ms/step - loss: 0.1981 - val_loss: 0.2235 - lr: 6.3025e-04
Epoch 22/200
18/18 [==============================] - ETA: 0s - loss: 0.1843
Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0005688000208465382.
18/18 [==============================] - 7s 374ms/step - loss: 0.1843 - val_loss: 0.2442 - lr: 5.9874e-04
Epoch 23/200
18/18 [==============================] - ETA: 0s - loss: 0.1934
Epoch 23: ReduceLROnPlateau reducing learning rate to 0.0005403600225690752.
18/18 [==============================] - 7s 376ms/step - loss: 0.1934 - val_loss: 0.2479 - lr: 5.6880e-04
Epoch 24/200
18/18 [==============================] - ETA: 0s - loss: 0.1930
Epoch 24: ReduceLROnPlat

18/18 [==============================] - 7s 375ms/step - loss: 0.1350 - val_loss: 0.2415 - lr: 1.3528e-04
Epoch 53/200
18/18 [==============================] - ETA: 0s - loss: 0.1383
Epoch 53: ReduceLROnPlateau reducing learning rate to 0.00012208651896798982.
18/18 [==============================] - 7s 371ms/step - loss: 0.1383 - val_loss: 0.2334 - lr: 1.2851e-04
Epoch 54/200
18/18 [==============================] - ETA: 0s - loss: 0.1358
Epoch 54: ReduceLROnPlateau reducing learning rate to 0.00011598219716688617.
18/18 [==============================] - 7s 371ms/step - loss: 0.1358 - val_loss: 0.2391 - lr: 1.2209e-04
Epoch 55/200
18/18 [==============================] - ETA: 0s - loss: 0.1172
Epoch 55: ReduceLROnPlateau reducing learning rate to 0.00011018308869097381.
18/18 [==============================] - 7s 368ms/step - loss: 0.1172 - val_loss: 0.2367 - lr: 1.1598e-04
Epoch 56/200
18/18 [==============================] - ETA: 0s - loss: 0.1290
Epoch 56: ReduceLROnPlateau reduci

18/18 [==============================] - ETA: 0s - loss: 0.1660
Epoch 35: ReduceLROnPlateau reducing learning rate to 0.00034056155709549785.
18/18 [==============================] - 7s 370ms/step - loss: 0.1660 - val_loss: 0.2452 - lr: 3.5849e-04
Epoch 36/200
18/18 [==============================] - ETA: 0s - loss: 0.1505
Epoch 36: ReduceLROnPlateau reducing learning rate to 0.00032353347924072293.
18/18 [==============================] - 7s 377ms/step - loss: 0.1505 - val_loss: 0.2313 - lr: 3.4056e-04
Epoch 37/200
18/18 [==============================] - ETA: 0s - loss: 0.1637
Epoch 37: ReduceLROnPlateau reducing learning rate to 0.00030735681357327847.
18/18 [==============================] - 7s 376ms/step - loss: 0.1637 - val_loss: 0.2044 - lr: 3.2353e-04
Epoch 38/200
18/18 [==============================] - ETA: 0s - loss: 0.1591
Epoch 38: ReduceLROnPlateau reducing learning rate to 0.00029198898118920624.
18/18 [==============================] - 7s 370ms/step - loss: 0.1591 - val

...

Epoch 53/200
18/18 [==============================] - ETA: 0s - loss: 0.1756
Epoch 53: ReduceLROnPlateau reducing learning rate to 0.00012851213177782482.
Restoring model weights from the end of the best epoch: 28.
18/18 [==============================] - 7s 395ms/step - loss: 0.1756 - val_loss: 0.2545 - lr: 1.3528e-04
Epoch 53: early stopping
2/2 [==============================] - 6s 127ms/step
0.2238, 0.2254, 0.4948
______fold 10______, ________repeat 7__________
Epoch 1/200
18/18 [==============================] - 53s 830ms/step - loss: 0.4653 - val_loss: 0.4371 - lr: 0.0010
Epoch 2/200
18/18 [==============================] - 7s 412ms/step - loss: 0.4490 - val_loss: 0.4198 - lr: 0.0010
Epoch 3/200
18/18 [==============================] - 7s 405ms/step - loss: 0.4202 - val_loss: 0.4111 - lr: 0.0010
Epoch 4/200
18/18 [==============================] - ETA: 0s - loss: 0.3893
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
18/18 [============================

18/18 [==============================] - ETA: 0s - loss: 0.1394
Epoch 77: ReduceLROnPlateau reducing learning rate to 8.099469014268834e-05.
18/18 [==============================] - 7s 377ms/step - loss: 0.1394 - val_loss: 0.2207 - lr: 8.5258e-05
Epoch 78/200
18/18 [==============================] - ETA: 0s - loss: 0.1321
Epoch 78: ReduceLROnPlateau reducing learning rate to 7.694495252508204e-05.
18/18 [==============================] - 7s 380ms/step - loss: 0.1321 - val_loss: 0.2297 - lr: 8.0995e-05
Epoch 79/200
18/18 [==============================] - ETA: 0s - loss: 0.1243
Epoch 79: ReduceLROnPlateau reducing learning rate to 7.309770662686787e-05.
18/18 [==============================] - 7s 370ms/step - loss: 0.1243 - val_loss: 0.2314 - lr: 7.6945e-05
Epoch 80/200
18/18 [==============================] - ETA: 0s - loss: 0.1321
Epoch 80: ReduceLROnPlateau reducing learning rate to 6.944281922187656e-05.
18/18 [==============================] - 7s 369ms/step - loss: 0.1321 - val_los

Restoring model weights from the end of the best epoch: 37.
18/18 [==============================] - 7s 392ms/step - loss: 0.1219 - val_loss: 0.2121 - lr: 1.1598e-04
Epoch 62: early stopping
2/2 [==============================] - 6s 121ms/step
0.1447, 0.1822, 0.3746
______fold 10______, ________repeat 9__________
Epoch 1/200
18/18 [==============================] - 53s 867ms/step - loss: 0.4986 - val_loss: 0.4486 - lr: 0.0010
Epoch 2/200
18/18 [==============================] - 7s 413ms/step - loss: 0.4664 - val_loss: 0.4415 - lr: 0.0010
Epoch 3/200
18/18 [==============================] - 7s 397ms/step - loss: 0.4544 - val_loss: 0.4354 - lr: 0.0010
Epoch 4/200
18/18 [==============================] - 7s 397ms/step - loss: 0.4376 - val_loss: 0.4221 - lr: 0.0010
Epoch 5/200
18/18 [==============================] - 7s 400ms/step - loss: 0.4115 - val_loss: 0.4161 - lr: 0.0010
Epoch 6/200
18/18 [==============================] - 7s 398ms/step - loss: 0.4172 - val_loss: 0.3852 - lr: 0.0010


Epoch 42/200
18/18 [==============================] - ETA: 0s - loss: 0.1696
Epoch 42: ReduceLROnPlateau reducing learning rate to 0.00030735681357327847.
18/18 [==============================] - 7s 369ms/step - loss: 0.1696 - val_loss: 0.1761 - lr: 3.2353e-04
Epoch 43/200
18/18 [==============================] - 7s 389ms/step - loss: 0.1603 - val_loss: 0.1438 - lr: 3.0736e-04
Epoch 44/200
18/18 [==============================] - ETA: 0s - loss: 0.1604
Epoch 44: ReduceLROnPlateau reducing learning rate to 0.00029198898118920624.
18/18 [==============================] - 7s 374ms/step - loss: 0.1604 - val_loss: 0.1702 - lr: 3.0736e-04
Epoch 45/200
18/18 [==============================] - ETA: 0s - loss: 0.1488
Epoch 45: ReduceLROnPlateau reducing learning rate to 0.00027738953212974593.
18/18 [==============================] - 7s 379ms/step - loss: 0.1488 - val_loss: 0.1945 - lr: 2.9199e-04
Epoch 46/200
18/18 [==============================] - ETA: 0s - loss: 0.1458
Epoch 46: ReduceLROnP

18/18 [==============================] - ETA: 0s - loss: 0.3297
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.0009025000152178108.
18/18 [==============================] - 7s 385ms/step - loss: 0.3297 - val_loss: 0.3221 - lr: 9.5000e-04
Epoch 10/200
18/18 [==============================] - 7s 402ms/step - loss: 0.3082 - val_loss: 0.3111 - lr: 9.0250e-04
Epoch 11/200
18/18 [==============================] - 7s 402ms/step - loss: 0.2805 - val_loss: 0.2951 - lr: 9.0250e-04
Epoch 12/200
18/18 [==============================] - 7s 405ms/step - loss: 0.2453 - val_loss: 0.2562 - lr: 9.0250e-04
Epoch 13/200
18/18 [==============================] - ETA: 0s - loss: 0.2528
Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0008573750033974647.
18/18 [==============================] - 8s 419ms/step - loss: 0.2528 - val_loss: 0.2563 - lr: 9.0250e-04
Epoch 14/200
18/18 [==============================] - ETA: 0s - loss: 0.2418
Epoch 14: ReduceLROnPlateau reducing learning rate to 0.0008